# AST paper

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/Thesis/ast
!pip install wget timm==0.4.5

Mounted at /content/drive
/content/drive/MyDrive/Thesis/ast
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 11.5 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=9ab3b85250fd244b3b41deda934a5de9ad2c0839df08f8bbaf1dd823560b30be
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
class Arguments():

  model='ast'
  dataset='speechcommands'
  imagenetpretrain=True
  audiosetpretrain=False

  bal=None
  lr=2.5e-4

  n_epochs=20
  freqm=48
  timem=48
  mixup=0.6
  batch_size=128
  fstride=10
  tstride=10
  dataset_mean=-6.845978
  dataset_std=5.5654526
  audio_length=128
  noise=True
  num_workers = 32
  exp_dir = '/content/drive/MyDrive/Thesis/resout'
  optimizer = 'adam'
  metrics='acc'
  loss='BCE'

  lrscheduler_start=5
  lrscheduler_step=1
  lrscheduler_decay=0.85

  warmup = False
  wa = False
  wa_start = 1
  wa_end = 5

  n_print_steps = 100
  n_class = 35
  lr_patience = 2
  save_model = True
args = Arguments()

In [ ]:
from src import dataloader
from src import models
from src.traintest import train, validate


audio_model = models.ASTModel(label_dim=args.n_class, fstride=args.fstride, tstride=args.tstride, input_fdim=128,
                                  input_tdim=args.audio_length, imagenet_pretrain=args.imagenetpretrain,
                                  audioset_pretrain=args.audiosetpretrain, model_size='base384')

---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: False
frequncey stride=10, time stride=10
number of patches=144


In [ ]:
import torch
data_train = '/content/drive/MyDrive/Thesis/datafiles/speechcommand_train_data.json'
data_val_path ='/content/drive/MyDrive/Thesis/datafiles/speechcommand_valid_data.json'
data_eval_path ='/content/drive/MyDrive/Thesis/datafiles/speechcommand_eval_data.json'


label_csv = '/content/drive/MyDrive/Thesis/ast/egs/speechcommands/data/speechcommands_class_labels_indices.csv'


audio_conf = {'num_mel_bins': 128, 'target_length': args.audio_length, 'freqm': args.freqm, 'timem': args.timem, 'mixup': args.mixup, 'dataset': args.dataset, 'mode':'train', 'mean':args.dataset_mean, 'std':args.dataset_std,
                  'noise':args.noise}

val_audio_conf = {'num_mel_bins': 128, 'target_length': args.audio_length, 'freqm': 0, 'timem': 0, 'mixup': 0, 'dataset': args.dataset, 'mode':'validation', 'mean':args.dataset_mean, 'std':args.dataset_std, 'noise':False}
eval_audio_conf = {'num_mel_bins': 128, 'target_length': args.audio_length, 'freqm': 0, 'timem': 0, 'mixup': 0, 'dataset': args.dataset, 'mode':'evaluation', 'mean':args.dataset_mean, 'std':args.dataset_std, 'noise':False}

train_loader = torch.utils.data.DataLoader(
            dataloader.AudiosetDataset(data_train, label_csv=label_csv, audio_conf=audio_conf),
            batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers, pin_memory=True)

eval_loader = torch.utils.data.DataLoader(
        dataloader.AudiosetDataset(data_eval_path, label_csv=label_csv, audio_conf=val_audio_conf),
        batch_size=args.batch_size*2, shuffle=False, num_workers=args.num_workers, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
        dataloader.AudiosetDataset(data_val_path, label_csv=label_csv, audio_conf=val_audio_conf),
        batch_size=args.batch_size*2, shuffle=False, num_workers=args.num_workers, pin_memory=True)

---------------the train dataloader---------------
now using following mask: 48 freq, 48 time
now using mix-up with rate 0.600000
now process speechcommands
use dataset mean -6.846 and std 5.565 to normalize the input.
now use noise augmentation
number of classes is 35


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


---------------the validation dataloader---------------
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process speechcommands
use dataset mean -6.846 and std 5.565 to normalize the input.
number of classes is 35
---------------the validation dataloader---------------
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process speechcommands
use dataset mean -6.846 and std 5.565 to normalize the input.
number of classes is 35


In [ ]:
%cd ..

/content/drive/MyDrive/Thesis


## Training

In [ ]:
print('Now starting training for {:d} epochs'.format(args.n_epochs))

train(audio_model, train_loader, val_loader, args)

Now starting training for 20 epochs
running on cuda
Total parameter number is : 86.933 million
Total trainable parameter number is : 86.933 million
now training with speechcommands, main metrics: acc, loss function: BCEWithLogitsLoss(), learning rate scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x7fcc58251ca0>
The learning rate scheduler starts at 5 epoch with decay rate of 0.850 every 1 epochs
current #steps=0, #epochs=1
start training...
---------------
2023-03-18 14:31:07.106484
current #epochs=1, #steps=0
Epoch: [1][100/662]	Per Sample Total Time 0.03168	Per Sample Data Time 0.03038	Per Sample DNN Time 0.00130	Train Loss 0.1380	
Epoch: [1][200/662]	Per Sample Total Time 0.02596	Per Sample Data Time 0.02477	Per Sample DNN Time 0.00120	Train Loss 0.1310	
Epoch: [1][300/662]	Per Sample Total Time 0.02296	Per Sample Data Time 0.02180	Per Sample DNN Time 0.00116	Train Loss 0.1211	
Epoch: [1][400/662]	Per Sample Total Time 0.02112	Per Sample Data Time 0.01998	Per Sample DNN

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: [2][38/662]	Per Sample Total Time 0.00241	Per Sample Data Time 0.00129	Per Sample DNN Time 0.00112	Train Loss 0.0678	
Epoch: [2][138/662]	Per Sample Total Time 0.00149	Per Sample Data Time 0.00036	Per Sample DNN Time 0.00113	Train Loss 0.0665	
Epoch: [2][238/662]	Per Sample Total Time 0.00134	Per Sample Data Time 0.00021	Per Sample DNN Time 0.00113	Train Loss 0.0657	
Epoch: [2][338/662]	Per Sample Total Time 0.00128	Per Sample Data Time 0.00015	Per Sample DNN Time 0.00113	Train Loss 0.0650	
Epoch: [2][438/662]	Per Sample Total Time 0.00124	Per Sample Data Time 0.00012	Per Sample DNN Time 0.00113	Train Loss 0.0644	
Epoch: [2][538/662]	Per Sample Total Time 0.00122	Per Sample Data Time 0.00010	Per Sample DNN Time 0.00112	Train Loss 0.0637	
Epoch: [2][638/662]	Per Sample Total Time 0.00120	Per Sample Data Time 0.00008	Per Sample DNN Time 0.00112	Train Loss 0.0632	
start validation
acc: 0.950606
AUC: 0.999203
Avg Precision: 0.490916
Avg Recall: 0.992908
d_prime: 4.464581
train_loss:

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: [3][76/662]	Per Sample Total Time 0.00175	Per Sample Data Time 0.00062	Per Sample DNN Time 0.00113	Train Loss 0.0594	
Epoch: [3][176/662]	Per Sample Total Time 0.00140	Per Sample Data Time 0.00027	Per Sample DNN Time 0.00113	Train Loss 0.0590	
Epoch: [3][276/662]	Per Sample Total Time 0.00130	Per Sample Data Time 0.00018	Per Sample DNN Time 0.00113	Train Loss 0.0590	
Epoch: [3][376/662]	Per Sample Total Time 0.00126	Per Sample Data Time 0.00013	Per Sample DNN Time 0.00113	Train Loss 0.0588	
Epoch: [3][476/662]	Per Sample Total Time 0.00123	Per Sample Data Time 0.00010	Per Sample DNN Time 0.00113	Train Loss 0.0584	
Epoch: [3][576/662]	Per Sample Total Time 0.00121	Per Sample Data Time 0.00009	Per Sample DNN Time 0.00113	Train Loss 0.0582	
start validation
acc: 0.959723
AUC: 0.999397
Avg Precision: 0.472862
Avg Recall: 0.995461
d_prime: 4.578748
train_loss: 0.058058
valid_loss: 0.009565
validation finished
Epoch-3 lr: 0.00025
epoch 3 training time: 120.512
---------------
2023-03-

...

Epoch: [18][46/662]	Per Sample Total Time 0.00217	Per Sample Data Time 0.00104	Per Sample DNN Time 0.00113	Train Loss 0.0421	
Epoch: [18][146/662]	Per Sample Total Time 0.00146	Per Sample Data Time 0.00033	Per Sample DNN Time 0.00113	Train Loss 0.0419	
Epoch: [18][246/662]	Per Sample Total Time 0.00133	Per Sample Data Time 0.00020	Per Sample DNN Time 0.00113	Train Loss 0.0415	
Epoch: [18][346/662]	Per Sample Total Time 0.00127	Per Sample Data Time 0.00014	Per Sample DNN Time 0.00113	Train Loss 0.0414	
Epoch: [18][446/662]	Per Sample Total Time 0.00124	Per Sample Data Time 0.00011	Per Sample DNN Time 0.00113	Train Loss 0.0414	
Epoch: [18][546/662]	Per Sample Total Time 0.00122	Per Sample Data Time 0.00009	Per Sample DNN Time 0.00113	Train Loss 0.0414	
Epoch: [18][646/662]	Per Sample Total Time 0.00120	Per Sample Data Time 0.00008	Per Sample DNN Time 0.00112	Train Loss 0.0414	
start validation
acc: 0.978359
AUC: 0.999680
Avg Precision: 0.604816
Avg Recall: 0.996333
d_prime: 4.828046
trai

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: [19][84/662]	Per Sample Total Time 0.00172	Per Sample Data Time 0.00059	Per Sample DNN Time 0.00113	Train Loss 0.0413	
Epoch: [19][184/662]	Per Sample Total Time 0.00140	Per Sample Data Time 0.00027	Per Sample DNN Time 0.00113	Train Loss 0.0409	
Epoch: [19][284/662]	Per Sample Total Time 0.00131	Per Sample Data Time 0.00018	Per Sample DNN Time 0.00113	Train Loss 0.0411	
Epoch: [19][384/662]	Per Sample Total Time 0.00126	Per Sample Data Time 0.00013	Per Sample DNN Time 0.00113	Train Loss 0.0411	
Epoch: [19][484/662]	Per Sample Total Time 0.00123	Per Sample Data Time 0.00011	Per Sample DNN Time 0.00113	Train Loss 0.0412	
Epoch: [19][584/662]	Per Sample Total Time 0.00122	Per Sample Data Time 0.00009	Per Sample DNN Time 0.00113	Train Loss 0.0411	
start validation
acc: 0.978359
AUC: 0.999646
Avg Precision: 0.532521
Avg Recall: 0.987917
d_prime: 4.788813
train_loss: 0.041091
valid_loss: 0.004604
validation finished
Epoch-19 lr: 2.1838554775312915e-05
epoch 19 training time: 118.642
-

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: [20][22/662]	Per Sample Total Time 0.00322	Per Sample Data Time 0.00208	Per Sample DNN Time 0.00115	Train Loss 0.0413	
Epoch: [20][122/662]	Per Sample Total Time 0.00153	Per Sample Data Time 0.00040	Per Sample DNN Time 0.00114	Train Loss 0.0410	
Epoch: [20][222/662]	Per Sample Total Time 0.00135	Per Sample Data Time 0.00022	Per Sample DNN Time 0.00113	Train Loss 0.0407	
Epoch: [20][322/662]	Per Sample Total Time 0.00128	Per Sample Data Time 0.00015	Per Sample DNN Time 0.00113	Train Loss 0.0407	
Epoch: [20][422/662]	Per Sample Total Time 0.00125	Per Sample Data Time 0.00012	Per Sample DNN Time 0.00113	Train Loss 0.0407	
Epoch: [20][522/662]	Per Sample Total Time 0.00122	Per Sample Data Time 0.00010	Per Sample DNN Time 0.00113	Train Loss 0.0407	
Epoch: [20][622/662]	Per Sample Total Time 0.00121	Per Sample Data Time 0.00008	Per Sample DNN Time 0.00112	Train Loss 0.0407	
start validation
acc: 0.978860
AUC: 0.999664
Avg Precision: 0.668341
Avg Recall: 0.995573
d_prime: 4.809884
trai

In [ ]:
args.exp_dir + '/models/best_audio_model.pth'

'/content/drive/MyDrive/Thesis/resout/models/best_audio_model.pth'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sd = torch.load(args.exp_dir + '/models/best_audio_model.pth', map_location=device)
audio_model = torch.nn.DataParallel(audio_model)
audio_model.load_state_dict(sd)

<All keys matched successfully>

## Valid

In [ ]:
import numpy as np
# best model on the validation set
stats, _ = validate(audio_model, val_loader, args, 'valid_set')
# note it is NOT mean of class-wise accuracy
val_acc = stats[0]['acc']
val_mAUC = np.mean([stat['auc'] for stat in stats])
print('---------------evaluate on the validation set---------------')
print("Accuracy: {:.6f}".format(val_acc))
print("AUC: {:.6f}".format(val_mAUC))


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


---------------evaluate on the validation set---------------
Accuracy: 0.978860
AUC: 0.999664


## Test set

In [ ]:
# test the model on the evaluation set
stats, _ = validate(audio_model, eval_loader, args, 'eval_set')
eval_acc = stats[0]['acc']
eval_mAUC = np.mean([stat['auc'] for stat in stats])
print('---------------evaluate on the test set---------------')
print("Accuracy: {:.6f}".format(eval_acc))
print("AUC: {:.6f}".format(eval_mAUC))
np.savetxt(args.exp_dir + '/eval_result.csv', [val_acc, val_mAUC, eval_acc, eval_mAUC])

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


---------------evaluate on the test set---------------
Accuracy: 0.981009
AUC: 0.999828


In [ ]:
# # check if all files exists in the folder
import json 
data_train = '/content/drive/MyDrive/Thesis/datafiles/speechcommand_train_data.json'
data_val_path ='/content/drive/MyDrive/Thesis/datafiles/speechcommand_valid_data.json'
data_eval_path ='/content/drive/MyDrive/Thesis/datafiles/speechcommand_eval_data.json'
f = open(data_train)
import pathlib
t_data = json.load(f)
counter = 0
total = 0
for i in t_data['data']:
  path = i['wav']

  if not pathlib.Path(path).is_file():
    print(path)
    counter+=1
  else:
    total+=1
  # break
print(counter)
print(total)

0
84707


# Remove pretrained from original: rebuild model

In [ ]:
args_new =  Arguments()
args_new.imagenetpretrain = False
args_new.exp_dir = '/content/drive/MyDrive/Thesis/resout_not_pretrained'

In [ ]:
!pwd


/content/drive/MyDrive/Thesis/ast


In [ ]:
# !mkdir /content/drive/MyDrive/Thesis/resout_not_pretrained
# !mkdir /content/drive/MyDrive/Thesis/resout_not_pretrained/predictions
# !mkdir /content/drive/MyDrive/Thesis/resout_not_pretrained/models
# !cp /content/drive/MyDrive/Thesis/resout_ast_original/predictions/target.csv /content/drive/MyDrive/Thesis/resout_not_pretrained/predictions


In [ ]:

audio_model_not_trained = models.ASTModel(label_dim=args_new.n_class, fstride=args_new.fstride, tstride=args_new.tstride, input_fdim=128,
                                  input_tdim=args_new.audio_length, imagenet_pretrain=args_new.imagenetpretrain,
                                  audioset_pretrain=args_new.audiosetpretrain, model_size='base384')

---------------AST Model Summary---------------
ImageNet pretraining: False, AudioSet pretraining: False
frequncey stride=10, time stride=10
number of patches=144


In [ ]:
%cd ..

/content/drive/MyDrive/Thesis


## Train

In [ ]:
print('Now starting training for {:d} epochs'.format(args_new.n_epochs))

train(audio_model_not_trained, train_loader, val_loader, args_new)

Now starting training for 20 epochs
running on cuda
Total parameter number is : 86.933 million
Total trainable parameter number is : 86.933 million
now training with speechcommands, main metrics: acc, loss function: BCEWithLogitsLoss(), learning rate scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x7febcfdca3b0>
The learning rate scheduler starts at 5 epoch with decay rate of 0.850 every 1 epochs
current #steps=0, #epochs=1
start training...
---------------
2023-05-08 11:23:41.474974
current #epochs=1, #steps=0


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: [1][100/662]	Per Sample Total Time 0.02320	Per Sample Data Time 0.02045	Per Sample DNN Time 0.00274	Train Loss 0.1376	
Epoch: [1][200/662]	Per Sample Total Time 0.01778	Per Sample Data Time 0.01544	Per Sample DNN Time 0.00234	Train Loss 0.1331	
Epoch: [1][300/662]	Per Sample Total Time 0.01557	Per Sample Data Time 0.01335	Per Sample DNN Time 0.00223	Train Loss 0.1316	
Epoch: [1][400/662]	Per Sample Total Time 0.01390	Per Sample Data Time 0.01174	Per Sample DNN Time 0.00216	Train Loss 0.1308	
Epoch: [1][500/662]	Per Sample Total Time 0.01274	Per Sample Data Time 0.01061	Per Sample DNN Time 0.00213	Train Loss 0.1303	
Epoch: [1][600/662]	Per Sample Total Time 0.01178	Per Sample Data Time 0.00967	Per Sample DNN Time 0.00211	Train Loss 0.1299	
start validation
acc: 0.058211
AUC: 0.644469
Avg Precision: 0.034659
Avg Recall: 0.976816
d_prime: 0.523867
train_loss: 0.129715
valid_loss: 0.125909
validation finished
Epoch-1 lr: 0.00025
epoch 1 training time: 1143.958
---------------
2023-0

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: [2][38/662]	Per Sample Total Time 0.01006	Per Sample Data Time 0.00804	Per Sample DNN Time 0.00201	Train Loss 0.1277	
Epoch: [2][138/662]	Per Sample Total Time 0.00604	Per Sample Data Time 0.00386	Per Sample DNN Time 0.00218	Train Loss 0.1273	
Epoch: [2][238/662]	Per Sample Total Time 0.00532	Per Sample Data Time 0.00310	Per Sample DNN Time 0.00222	Train Loss 0.1269	
Epoch: [2][338/662]	Per Sample Total Time 0.00499	Per Sample Data Time 0.00275	Per Sample DNN Time 0.00224	Train Loss 0.1266	
Epoch: [2][438/662]	Per Sample Total Time 0.00482	Per Sample Data Time 0.00257	Per Sample DNN Time 0.00225	Train Loss 0.1264	
Epoch: [2][538/662]	Per Sample Total Time 0.00472	Per Sample Data Time 0.00247	Per Sample DNN Time 0.00225	Train Loss 0.1261	
Epoch: [2][638/662]	Per Sample Total Time 0.00463	Per Sample Data Time 0.00239	Per Sample DNN Time 0.00225	Train Loss 0.1259	
start validation
acc: 0.133955
AUC: 0.768913
Avg Precision: 0.103267
Avg Recall: 0.533425
d_prime: 1.039831
train_loss:

...

Epoch: [18][46/662]	Per Sample Total Time 0.00914	Per Sample Data Time 0.00709	Per Sample DNN Time 0.00206	Train Loss 0.0811	
Epoch: [18][146/662]	Per Sample Total Time 0.00617	Per Sample Data Time 0.00397	Per Sample DNN Time 0.00220	Train Loss 0.0813	
Epoch: [18][246/662]	Per Sample Total Time 0.00544	Per Sample Data Time 0.00327	Per Sample DNN Time 0.00217	Train Loss 0.0814	
Epoch: [18][346/662]	Per Sample Total Time 0.00515	Per Sample Data Time 0.00300	Per Sample DNN Time 0.00214	Train Loss 0.0815	
Epoch: [18][446/662]	Per Sample Total Time 0.00499	Per Sample Data Time 0.00284	Per Sample DNN Time 0.00215	Train Loss 0.0813	
Epoch: [18][546/662]	Per Sample Total Time 0.00503	Per Sample Data Time 0.00286	Per Sample DNN Time 0.00217	Train Loss 0.0813	
Epoch: [18][646/662]	Per Sample Total Time 0.00484	Per Sample Data Time 0.00266	Per Sample DNN Time 0.00218	Train Loss 0.0814	
start validation
acc: 0.894800
AUC: 0.996394
Avg Precision: 0.181793
Avg Recall: 0.995170
d_prime: 3.799813
trai

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: [19][84/662]	Per Sample Total Time 0.00721	Per Sample Data Time 0.00502	Per Sample DNN Time 0.00218	Train Loss 0.0812	
Epoch: [19][184/662]	Per Sample Total Time 0.00581	Per Sample Data Time 0.00366	Per Sample DNN Time 0.00215	Train Loss 0.0812	
Epoch: [19][284/662]	Per Sample Total Time 0.00538	Per Sample Data Time 0.00321	Per Sample DNN Time 0.00217	Train Loss 0.0812	
Epoch: [19][384/662]	Per Sample Total Time 0.00531	Per Sample Data Time 0.00312	Per Sample DNN Time 0.00219	Train Loss 0.0811	
Epoch: [19][484/662]	Per Sample Total Time 0.00522	Per Sample Data Time 0.00304	Per Sample DNN Time 0.00218	Train Loss 0.0810	
Epoch: [19][584/662]	Per Sample Total Time 0.00512	Per Sample Data Time 0.00293	Per Sample DNN Time 0.00219	Train Loss 0.0809	
start validation
acc: 0.898006
AUC: 0.996388
Avg Precision: 0.176084
Avg Recall: 0.995335
d_prime: 3.799087
train_loss: 0.080831
valid_loss: 0.020805
validation finished
Epoch-19 lr: 2.1838554775312915e-05
epoch 19 training time: 463.285
-

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: [20][22/662]	Per Sample Total Time 0.01183	Per Sample Data Time 0.00972	Per Sample DNN Time 0.00211	Train Loss 0.0788	
Epoch: [20][122/662]	Per Sample Total Time 0.00648	Per Sample Data Time 0.00430	Per Sample DNN Time 0.00218	Train Loss 0.0804	
Epoch: [20][222/662]	Per Sample Total Time 0.00583	Per Sample Data Time 0.00363	Per Sample DNN Time 0.00220	Train Loss 0.0802	
Epoch: [20][322/662]	Per Sample Total Time 0.00581	Per Sample Data Time 0.00358	Per Sample DNN Time 0.00223	Train Loss 0.0804	
Epoch: [20][422/662]	Per Sample Total Time 0.00552	Per Sample Data Time 0.00331	Per Sample DNN Time 0.00221	Train Loss 0.0802	
Epoch: [20][522/662]	Per Sample Total Time 0.00535	Per Sample Data Time 0.00312	Per Sample DNN Time 0.00224	Train Loss 0.0802	
Epoch: [20][622/662]	Per Sample Total Time 0.00525	Per Sample Data Time 0.00302	Per Sample DNN Time 0.00223	Train Loss 0.0803	
start validation
acc: 0.896604
AUC: 0.996462
Avg Precision: 0.171731
Avg Recall: 0.996073
d_prime: 3.808864
trai

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
audio_model_not_trained = torch.nn.DataParallel(audio_model_not_trained)
audio_model_not_trained.load_state_dict(torch.load(args_new.exp_dir + '/models/best_audio_model.pth', map_location=device))

## Validation

In [ ]:
import numpy as np
# best model on the validation set
stats, _ = validate(audio_model_not_trained, val_loader, args_new, 'valid_set')
# note it is NOT mean of class-wise accuracy
val_acc = stats[0]['acc']
val_mAUC = np.mean([stat['auc'] for stat in stats])
print('---------------evaluate on the validation set---------------')
print("Accuracy: {:.6f}".format(val_acc))
print("AUC: {:.6f}".format(val_mAUC))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


---------------evaluate on the validation set---------------
Accuracy: 0.898006
AUC: 0.996388


## Test

In [ ]:
# test the model on the evaluation set
stats, _ = validate(audio_model_not_trained, eval_loader, args_new, 'eval_set')
eval_acc = stats[0]['acc']
eval_mAUC = np.mean([stat['auc'] for stat in stats])
print('---------------evaluate on the test set---------------')
print("Accuracy: {:.6f}".format(eval_acc))
print("AUC: {:.6f}".format(eval_mAUC))
np.savetxt(args_new.exp_dir + '/eval_result.csv', [val_acc, val_mAUC, eval_acc, eval_mAUC])

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


---------------evaluate on the test set---------------
Accuracy: 0.876874
AUC: 0.995664
